# 합성곱 신경망(CNN)


-  이미지 인식, 음성 인식 등 다양한 곳에서 사용 됨

# 1. 전체 구조

- 합성곱 계층(convolutional layer)과 풀링 계층(pooling layer)이 새로 나옴
- 앞에서 본 신경망은 인접하는 모든 뉴런과 결합되어 있었다.(Affine 계층, fully-connected{완전연결})
- CNN도 출력 계층에서는 Affine-Softmax 조합을 보통 사용한다.


![cnn_fcn_비교](./images/cnn_fcn.png)

# 2. 합성곱 계층

- 패딩(padding), 스트라이드(stride)라는 용어가 등장

## 2.1. 완전연결 계층의 문제점

- 데이터의 형상이 무시된다. -> 그림의 경우 가로, 세로, 색상의 3차원이지만 완전연결을 사용하면 1차원으로 변환하여 사용하게 된다. -> 공간적 형상이 무시된다.

- 합성곱 계층은 형상을 유지한다. -> 3차원 데이터를 입력 받으면, 3차원 데이터로 전달한다.


### 합성곱 계층망의 용어
- 입출력 데이터 -> 특징 맵(feature map) (입력 데이터 : 입력 특징 맵, 출력 데이터: 출력 특징 맵)

## 2.2. 합성곱 연산

- 합성곱 연산 : 이미지 처리에서 말하는 필터 연산

- 합성곱 연산 예시 그림
![합성곱_연산](./images/cnn_cal.png)

- 위의 입력데이터는 높이: 4, 너비: 4이며 이 형상을 (4,4)로 표시한다.
- 위와 같이 필터는 (3,3), 출력 맵은 (2,2)가 된다.
- 필터는 커널이라고 칭하기도 한다.


### 합성곱 계산
- 입력 데이터에 필터의 윈도우를 적용하며, 위도우를 이동시키면서 연산한다.
- 윈도우와 입력데이터의 대응하는 원소끼리 곱하여 총합을 출력 맵에 저장한다.


- 합성곱 연산의 그림 예
![합성곱_연산의_예](./images/cnn_cal_ex.png)

- CNN에서는 필터가 가중치에 해당한다.
- 편향을 추가하면 아래와 같은 그림처럼 진행된다.
![합성곱_연산의_편향_예](./images/cnn_cal_ex2.png)

## 2.3. 패딩(Padding)

- 합성곱 연산 전에 입력 데이터 주변을 특정 값으로 채우는 것


- (4,4) 크기의 입력데이터에 폭이 1이고 값이 0인 패딩을 채우는 예
![패딩의_예](./images/padding.png)
패딩이 추가되어 (4,4)가 (6,6)으로 변한다.

## 2.4. 스트라이드

- 필터를 적용하는 간격을 의미
- 예를 들어서 스트라이드가 2라면 필터를 적용하는 윈도우가 두 칸 씩 이동함.


![패딩의_예](./images/stride.png)

### 패딩과 스트라이드를 가지고 출력 크기 계산하기

- 입력 크기를(H,W), 필터 크기를(FH, FW), 출력 크기를(OH, OW), 패딩을 P, 스트라이드를 S라고 하면

$$
OH = \frac{H + 2P - FH}{S} + 1 \\
OW = \frac{W + 2P - FW}{S} + 1
$$

## 2.5. 3차원 데이터의 합성곱 연산

- 3차원 데이터에서는 채널 방향으로 필터의 특징 맵이 늘어난다.
- 3차원 합성곱의 예시 사진
![3차원_합성곱](./images/3CNN_cal.png)

- 주의할 점으로는 입력 데이터의 채널 수와 필터의 채널 수가 같아야 한다.

# 3. 풀링 계층

- 세로, 가로 방향의 공간을 줄이는 연산
- 풀링의 예)
![최대_풀링_처리_순서](./images/max_pooling.png)

- 위 예시는 stride가 2인 최대 풀링(Max pooling)이다.
- 최대 풀링 외에도 평균 풀링같은 다른 풀링 방법이 있다.

## 3.1. 풀링 계층의 특징

1. 학습해야 할 매개변수가 없다.
    - 대상 영역의 최대값이나 평균을 취하는 처리이므로 학습할 것이 없다.

2. 채널 수가 변하지 않는다.
    - 입력 데이터의 채널 수 그대로 출력에 보내므로 채널 수가 변하지 않는다.
    
3. 입력의 변화에 영향을 적게 받는다
    - 입력 데이터가 조금 변해도 풀링의 결과는 잘 변하지 않는다.

# 4. 합성곱/풀링 계층 구현하기

## 4.1. 4차원 배열

- 데이터가 10개인 높이 28, 너비 28, 채널 1인 데이터라면 (10, 1, 28, 28)로 표시한다.

In [1]:
import numpy as np

x = np.random.rand(10, 1, 28, 28)
print("x의 형상: {}".format(x.shape))

x의 형상: (10, 1, 28, 28)


## 4.2. 합성곱 계층 구현하기

In [2]:
from common.util import im2col

# im2col을 이용하여 합성곱을 할 부위만큼을 2차원으로 만들어서 펼치는 작업을 보여줌
x1 = np.random.rand(1,3,7,7)
col1 = im2col(x1,5,5,stride=1, pad=0)
print(col1.shape)

(9, 75)


In [3]:
x2 = np.random.rand(10,3,7,7)
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(90, 75)


In [5]:
# 합성곱 계층 구현

class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride) # 출력의 크기 공식에 의해서
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)
        
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T
        out = np.dot(col, col_W) + self.b
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0,3,1,2)
        
        return out

In [7]:
# 풀링 계층 구현

class Pooling:
    def __init__(self, pool_h, pool_w, stride= 1, pad=0):
        sefl.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)
        
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)
        
        out = np.max(col, axis=1)
        
        out = out.reshape(N, out_h, out_w, C).transpose(0,3,1,2)
        
        return out

# 5. CNN 구현하기

- 사용하는 인자
    1. input_dim = 입력 데이터(채널 수, 높이, 너비)
    2. conv_param = 합성곱 계층의 하이퍼파라미터(딕셔너리)
        - 딕셔너리의 키
        1. filter_num = 필터 수
        2. filter_size = 필터 크기
        3. stride = 스트라이드
        4. pad = 패딩
        5. hidden_size = 은닉층의 뉴런 수
        6. output_size = 출력층의 뉴런 수
        7. weight_init_std = 초기화 때의 가중치 표준편차

In [10]:
class SimpleConvNet:
    def __init__(self, input_dim=(1, 28, 28),
                conv_param={'filter_num':30, 'filter_size': 5,
                           'pad':0, 'stride': 1},
                hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = cov_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))
        
        
        # 가중치 매개변수 구현
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], 
                                                              filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.parmas['W2'] = weight_init_std * np.random.randn(pool_output_size,
                                                             hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)
        
        # CNN 구성하는 계층구현
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'],
                                          self.params['b1'],
                                          conv_param['stride'],
                                          conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'],
                                       self.params['b2'])
        self.layers['Relu2'] = Relu()